In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
loader = PyPDFLoader("2312.13382.pdf")
docs = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [7]:
groq_api_key = ""

In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1",
    encode_kwargs = {'precision': 'binary'}
    )
one_bit_vectorstore = FAISS.from_documents(documents, embeddings)
one_bit_retriever = one_bit_vectorstore.as_retriever(
                                                    search_kwargs={"k": 3}
                                                    )

In [9]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")
model = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

In [10]:
document_chain = create_stuff_documents_chain(model, prompt)
one_bit_retrieval_chain = create_retrieval_chain(one_bit_retriever, document_chain)

In [11]:
%%time
response = one_bit_retrieval_chain.invoke({"input": "What does the paper introduce?"})
print(response["answer"])

The paper introduces DSPy, a system that enables generating harder few-shot examples to teach language models (LMs) to conduct challenging steps. It also presents a method called counterexample bootstrapping, which involves developing demonstrations that contain failed examples and traces to fix errors. These counterexamples are mixed with bootstrapped few-shot examples to help the LM avoid making the same mistakes without assertion-driven backtracking. Additionally, the paper discusses DSPy Assertions, which check for the correct answer's inclusion and verify the plausibility of answer choices.

The paper also presents a new task called TweetGen, which generates tweets to answer questions derived from the HotPotQA data set. The TweetGen system includes a DSPy program with LM Assertions to ensure the generated tweets meet certain criteria, such as not having hashtags, including the correct answer, being within a character limit, being engaging, and being faithful to the context. The pa

In [12]:
embeddings = HuggingFaceEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1"
    )
normal_vectorstore = FAISS.from_documents(documents, embeddings)
normal_retriever = normal_vectorstore.as_retriever(
                                                    search_kwargs={"k": 3}
                                                    )

In [13]:
normal_retrieval_chain = create_retrieval_chain(normal_retriever, document_chain)

In [14]:
%%time
response = normal_retrieval_chain.invoke({"input": "What is introduced in the paper?"})
print(response["answer"])

The paper introduces several concepts and tools related to language model pipelines. Here are the main ones:

1. DSPy: A framework for building language model pipelines that can handle complex tasks such as long-form question answering. It includes modules such as `dspy.Retrieve` for retrieving relevant passages and `dspy.ChainOfThought` for generating a series of reasoning steps.
2. LongFormQAWithAssertions: A class in DSPy that implements a long-form question answering pipeline with two assertions: (1) every 1-2 sentences should have a citation, and (2) the text preceding a citation should be faithful to the cited reference. These assertions are implemented using `dspy.Suggest` statements, which can provide suggestions or warnings to the user.
3. Citation Faithfulness: A metric for assessing the intrinsic performance of a language model pipeline. It checks if the text preceding each citation supports the cited context appropriately.
4. QuizGen: A new task introduced in the paper, whi

In [15]:
one_bit_vectorstore.save_local("one-bit")
normal_vectorstore.save_local("usual")